# **In this colab we are going to download all the nessesary files**

# **Step 1 – Check GPU configurations working properly**
First of all we need to check the GPU configurations working properly. For that please execute following piece of python code in the colab file.

In [0]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
 
with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)
   
sess = tf.Session(config=config)
 
try:
  sess.run(tf.global_variables_initializer())
  print('ok')
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
ok


# **Step 2 – Connect to the Google Drive**
As the next step, we need to connect our colab file to the google drive. This is because, if we going to train a new model we can save it in the google drive rather than save it in the temporary location provide by the colab. To do that, please execute following code in a new code widget on colab file. Before doing this, create a new folder in your google drive

In [0]:
from os.path import join
from google.colab import drive
 
ROOT = "/content/drive"
drive.mount(ROOT)

# This is a custom path. 'My Drive' is the root of your google drive change accordingly
PROJ = "My Drive/workshop" 
PROJECT_PATH = join(ROOT, PROJ)
 
#%cd ~/content 
%cd {PROJECT_PATH}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/workshop


# **Step 3 – Clone the Tensorflow models repository**
In this step, you can clone the github folder of Tensorflow´s Object detection API. Once completed you can check those files by navigating to Google drive -> My Drive -> your/folder/route directory.

**do it only once if the folder already has the models downloaded**

In [0]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 33957 (delta 5), reused 21 (delta 1), pack-reused 33928
Receiving objects: 100% (33957/33957), 512.22 MiB | 22.82 MiB/s, done.
Resolving deltas: 100% (21738/21738), done.
Checking out files: 100% (3173/3173), done.


# **Step 4 – Download base model**

Download the base model into the object_detection folder

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

**this has to be done only once **




In [0]:
ENV = "models/research/object_detection" 
PROJECT_ENV = join(PROJECT_PATH, ENV)
%cd {ENV}

import os
import shutil
import glob
import urllib.request
import tarfile

MODEL = 'faster_rcnn_inception_v2_coco_2018_01_28'
#MODEL = 'ssd_mobilenet_v1_coco_2018_01_28'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
#http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
DEST_DIR = '/content/models/research/object_detection'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)


/content/drive/My Drive/workshop/models/research/object_detection


# **Step 5 - Move example files from the object_detection_workshop folder**

we will need to move the following files & folders into the object detection folder
(/content/drive/My Drive/workshop/models/research/object_detection)


*   Images (folder containing sample images and its anotations)
*   inference graph (empty folder to save our inference graph model)
*   training folder (folder containing the config pipeline adn labelmap)
*   test1 (sample image file to test the model)
*   xml_to_csv.py (python code to collect all anotations in one csv file)
*   generate_tfrecord.py (pythone code to transform the csv file into a tfrecord)



In [0]:
from os.path import join
ENV = "/models/research/object_detection" 
PROJECT_ENV = join(PROJECT_PATH, ENV)
!cd {PROJECT_ENV}

import os
cwd = os.getcwd()
print (cwd)

!ls

NameError: ignored

# **Step 6 - Create a dataset**

A dataset must have 2 things


1.   Mixed sample images from the objects that we want to detect
2.   Anotations of the images

There are many tools that we can use to label our images, in this case we will use

https://github.com/tzutalin/labelImg

For windows you can download it from this link

https://drive.google.com/file/d/1Lnju0mDLB_nKdpSo4N1YdrxLGYLy7OJm/view?usp=sharing

